# Assignment: Text Processing with Encoding Techniques (10 points)

**Background:**
Building on basic text processing concepts, this assignment explores three fundamental encoding techniques for NLP: one-hot encoding, bag-of-words (BoW), and TF-IDF. You'll implement these methods on text data and analyze their differences.

**Note on generative AI use**: You are allowed to use any generative AI tool of your preference, as long as you are explicit about how and when/where you have used it to obtain your answers. You will be graded on the quality of your answers and your understanding of the concepts. NOT on the quality of your code. For example, you can use AI to come up with code solutions for each task, but a detailed explanation of your choices must be given. This includes but is not limited to: (i) what the code does, (ii) output samples if available, and (iii) your analysis. If your code is generated with AI, you should mention it and the name of the AI model used.

## **Instructions and Point Breakdown**

**1. Dataset Preparation (2 points)**

- Select a text dataset (e.g., news headlines, product reviews, tweets)
- Preprocess the text by:
    - Converting to lowercase
    - Removing punctuation/stopwords
    - Tokenizing sentences
- Briefly justify your dataset choice and any particular challenges you faced.

**2. Implement Encoding Techniques (4 points)**

- **One-Hot Encoding (1.5 pts):**
    - Build a vocabulary from unique tokens
    - Convert sample sentences to one-hot vectors
    - Use manual implementation (e.g., dictionary mapping)

- **Bag-of-Words (1.5 pts):**
    - Implement using `CountVectorizer`
    - Show document-term matrix
    - Compare with one-hot results

- **TF-IDF (1 pt):**
    - Generate TF-IDF vectors using `TfidfVectorizer`
    - Explain TF vs. IDF components

**3. Analysis and Visualization (2 points)**

- Perform **one** of:
    - Compare the top 5 features from each method
    - Visualize word importance differences (e.g., bar chart)
    - Compute cosine similarity between document vectors
- Describe whichever analysis you perform

**4. Technical Reflection (2 points)**

- In a Markdown cell:
    - Contrast use cases for each technique
    - Explain how TF-IDF solves BoW limitations
    - Discuss one challenge with sparse representations
    - Suggest an AI prompt for handling large vocabularies


## **Submission Requirements**

- Jupyter Notebook containing:
    - Code with comments (Sections 1-3)
    - Outputs including matrices/visualizations
    - Reflection (Section 4)
- Use Python libraries: `sklearn`, `pandas`, `matplotlib`

**Grading Rubric**


| Section | Points |
| :-- | :-- |
| Dataset prep \& preprocessing | 2 |
| One-hot implementation | 1.5 |
| BoW/TF-IDF implementation | 2.5 |
| Analysis/visualization | 2 |
| Quality of reflection | 2 |
| **Total** | **10** |

# *Sources used*
* https://github.com/opengeos/geospatial-data-catalogs
* https://www.geeksforgeeks.org/pandas/pandas-access-columns/
* https://www.geeksforgeeks.org/python/difference-between-map-applymap-and-apply-methods-in-pandas/
* https://www.geeksforgeeks.org/python/how-to-compare-two-dataframes-with-pandas-compare/
* https://www.geeksforgeeks.org/pandas/python-pandas-dataframe-sum/
* https://www.geeksforgeeks.org/pandas/python-pandas-series-nlargest/

# **Installs & Imports**

In [1]:
!pip install nltk -q

In [2]:
from typing import List
# Data preprocessing Libraries
import pandas as pd
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords # To remove stopwords in NLTK
nltk_stopwords = set(stopwords.words("english")) # Load English stopwords once for efficiency

# Libraries
from sklearn.feature_extraction.text import CountVectorizer # To create one-hot encoding & bag of words
from sklearn.feature_extraction.text import TfidfVectorizer # To create TF-IDF

[nltk_data] Downloading package stopwords to /home/mch84/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **1. Data Preparation (2 points)**
I chose this dataset because I wanted to learn more about how to do natural language processing on geospatial-related data. The dataset's source also contained concise instructions on how to retrieve the dataset. I didn't encounter particular challenges in this step, I mainly referred back to the previous 101 or 102 practice tutorial jupyter notebooks if I forgot something.

In [4]:
# Select text dataset
url = 'https://github.com/opengeos/geospatial-data-catalogs/raw/master/nasa_cmr_catalog.tsv'
dataframe = pd.read_csv(url, sep='\t')
title_description_dataframe = dataframe[["title", "description"]]
title_description_dataframe.head()

# Preprocess text by removing punctuation, extra whitespace, and stopwords.
def preprocess_text(text: str) -> str:
    text = text.lower() # Lowercase all text.
    text = re.sub(r"[^\w\s]", "", text) # Remove punctuation
    text = re.sub(r"\s+", " ", text) # remove extra whitespace
    text_no_stopwords = [token for token in text.split(" ") if token not in nltk_stopwords] # Store only the non-stopwords
    text_no_stopwords = " ".join(text_no_stopwords)
    return text_no_stopwords

preprocessed_dataframe = title_description_dataframe.copy() # Make a copy of original dataframe
preprocessed_dataframe["description"] = title_description_dataframe["description"].map(preprocess_text) # Preprocess the copy's data

# Tokenization & stopwords
print("Original:", title_description_dataframe["description"][0])
print("\nPreprocessed:", preprocessed_dataframe["description"][0])

Original: Indian Remote Sensing satellites (IRS) are a series of Earth Observation satellites, built, launched and maintained by Indian Space Research Organisation. The IRS series provides many remote sensing services to India and international ground stations. With 5 m resolution and products covering areas up to 70 km x 70 km IRS LISS-IV mono data provide a cost effective solution for mapping tasks up to 1:25'000 scale.

Preprocessed: indian remote sensing satellites irs series earth observation satellites built launched maintained indian space research organisation irs series provides many remote sensing services india international ground stations 5 resolution products covering areas 70 km x 70 km irs lissiv mono data provide cost effective solution mapping tasks 125000 scale


# **2. Implement Encoding Techniques (4 points)**

*TF vs. IDF components*:
*TF-IDF (Term Frequency-Inverse Document Frequency)*: Used weighting scheme to highlight important (rare) words in documents. Reduces impact of common words. Widely used for information retrieval & classification. Used in search engines.
>**Formula for Term Frequency** = # of word’s occurrence in text/total # of words in text = probability

>**Formula for Inverse Document Frequency** = idf(t) = log(1+N1+df(t))+1. Where N = total number of documents. df(t) = # of documents containing term t.

> **Formula for TF-IDF** = tf(t,d) * idf(t)


In [5]:
# Extract unique words to form vocabulary for each text
descriptions = preprocessed_dataframe["description"].tolist()[:10] # Get list of all description row values
vectorizer_vocab = CountVectorizer()
vectorizer_vocab.fit(descriptions)
vocab = vectorizer_vocab.get_feature_names_out()

# One-hot encoding
vectorizer = CountVectorizer(vocabulary=vocab, binary=True)
onehot = vectorizer.transform(descriptions)
onehot_dataframe = pd.DataFrame(onehot.toarray(),
                      columns=vocab) # Display frequency of each word in the given text through Pandas DataFrame
print("one-hot encoding dataframe:")
display(onehot_dataframe.head())

# Bag of Words
vectorizer = CountVectorizer()
vectorizer.fit(descriptions) # Fit vectorizer to list of text to build vocabulary
bag_of_words = vectorizer.transform(descriptions) # Transform list of text into a bag-of-words matrix
bow_dataframe = pd.DataFrame(bag_of_words.toarray(),
                      columns=vocab) # Display frequency of each word in the given text through Pandas DataFrame
print("\n\nBag of words dataframe:")
display(bow_dataframe.head())

# TF-IDF
vectorizer = TfidfVectorizer()
vectorizer.fit(descriptions) # Fit vectorizer to list of text to build vocabulary
tfidf_matrix = vectorizer.transform(descriptions) # Transform the documents into a TF-IDF-weighted term-document matrix
tfidf_dataframe = pd.DataFrame(tfidf_matrix.toarray(),
                      columns=vocab) # Display frequency of each word in the given text through Pandas DataFrame
print("\n\nTF-IDF matrix dataframe:")
display(tfidf_dataframe.head())

one-hot encoding dataframe:


,000017966259,00344257,025,101109tgrs20172734070,11,113,125000,190pp207216,19781101,19822016,...,windsat,within,work,wã¼rzler,year,yearly,yy,yyyymmdd,zircon,âimplications
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0




Bag of words dataframe:


,000017966259,00344257,025,101109tgrs20172734070,11,113,125000,190pp207216,19781101,19822016,...,windsat,within,work,wã¼rzler,year,yearly,yy,yyyymmdd,zircon,âimplications
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0




TF-IDF matrix dataframe:


,000017966259,00344257,025,101109tgrs20172734070,11,113,125000,190pp207216,19781101,19822016,...,windsat,within,work,wã¼rzler,year,yearly,yy,yyyymmdd,zircon,âimplications
0,0.0,0.0,0.0,0.0,0.0,0.0,0.133695,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.049577,...,0.0,0.042145,0.0,0.049577,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.056143,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.073856,0.0,0.000000,...,0.0,0.062785,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.073856
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000


In [6]:
# Compare one-hot encoding with bag of words encoding
print("\nOnehot = self. BoW = other. NaN = same values")
display(onehot_dataframe.compare(bow_dataframe))


Onehot = self. BoW = other. NaN = same values


2017       2018         70       absorption       aerosol        ...  \
  self other self other self other       self other    self other  ...   
0  NaN   NaN  NaN   NaN  1.0   2.0        NaN   NaN     NaN   NaN  ...   
1  NaN   NaN  1.0   2.0  NaN   NaN        NaN   NaN     NaN   NaN  ...   
2  NaN   NaN  NaN   NaN  NaN   NaN        1.0   6.0     NaN   NaN  ...   
3  NaN   NaN  NaN   NaN  NaN   NaN        NaN   NaN     NaN   NaN  ...   
4  NaN   NaN  NaN   NaN  NaN   NaN        NaN   NaN     1.0   3.0  ...   
5  1.0   3.0  NaN   NaN  NaN   NaN        NaN   NaN     NaN   NaN  ...   
6  NaN   NaN  NaN   NaN  NaN   NaN        NaN   NaN     NaN   NaN  ...   
7  NaN   NaN  NaN   NaN  NaN   NaN        1.0   3.0     NaN   NaN  ...   
8  NaN   NaN  NaN   NaN  NaN   NaN        NaN   NaN     NaN   NaN  ...   
9  NaN   NaN  NaN   NaN  NaN   NaN        NaN   NaN     NaN   NaN  ...   

  transantarctic        upb       used       wagner       zircon        
            self other self other self other   self other   self other  
0            NaN   NaN  NaN   NaN  NaN   NaN    NaN   NaN    NaN   NaN  
1            NaN   NaN  NaN   NaN  NaN   NaN    NaN   NaN    NaN   NaN  
2            NaN   NaN  NaN   NaN  NaN   NaN    NaN   NaN    NaN   NaN  
3            NaN   NaN  NaN   NaN  1.0   2.0    NaN   NaN    NaN   NaN  
4            NaN   NaN  NaN   NaN  NaN   NaN    NaN   NaN    NaN   NaN  
5            NaN   NaN  NaN   NaN  NaN   NaN    1.0   3.0    NaN   NaN  
6            NaN   NaN  NaN   NaN  NaN   NaN    NaN   NaN    NaN   NaN  
7            NaN   NaN  NaN   NaN  NaN   NaN    NaN   NaN    NaN   NaN  
8            1.0   2.0  1.0   3.0  NaN   NaN    NaN   NaN    1.0   4.0  
9            1.0   2.0  1.0   3.0  NaN   NaN    NaN   NaN    1.0   4.0  

[10 rows x 258 columns]

# **3. Analysis and Visualization (2 points)**

I choose to compare the top 5 features (words) from each method.


In [7]:
# five largest values in first 10 rows word columns across 3 dataframes
def display_top_n_features(dataframe, n=5):
    column_counts = dataframe.sum(axis=0) # sum the totals in each column (rows with more 1s/larger values will have larger sums)
    top_5_columns = column_counts.nlargest(n).index # Get indicies of top 5 columns with max sums
    display(dataframe[top_5_columns])
print("TOP 5 WORDS FROM EACH METHOD\n")
print("One hot encoding:")
display_top_n_features(onehot_dataframe)

print("\n\nBag of words:")
display_top_n_features(bow_dataframe)

print("\n\nTF-IDF:")
display_top_n_features(tfidf_dataframe)

TOP 5 WORDS FROM EACH METHOD

One hot encoding:


,data,also,esa,products,project
0,1,0,0,1,0
1,1,1,1,1,1
2,1,1,1,1,1
3,1,0,1,0,1
4,1,0,1,1,1
5,1,1,1,1,1
6,1,1,1,1,1
7,1,1,1,1,1
8,0,1,0,0,0
9,0,1,0,0,0




Bag of words:


,data,dataset,samples,also,cloud
0,1,0,0,0,0
1,5,9,0,2,13
2,5,4,0,2,0
3,4,0,0,0,0
4,1,1,0,0,0
5,3,2,0,2,0
6,3,4,0,2,0
7,4,2,0,3,0
8,0,0,7,1,0
9,0,0,7,1,0




TF-IDF:


,data,samples,dataset,cloud,products
0,0.059349,0.000000,0.000000,0.000000,0.065171
1,0.110039,0.000000,0.239528,0.644499,0.024167
2,0.146586,0.000000,0.141815,0.000000,0.032193
3,0.131143,0.000000,0.000000,0.000000,0.000000
4,0.073153,0.000000,0.088465,0.000000,0.240988
5,0.071781,0.000000,0.057871,0.000000,0.052549
6,0.157849,0.000000,0.254519,0.000000,0.115556
7,0.170128,0.000000,0.102869,0.000000,0.093409
8,0.000000,0.445671,0.000000,0.000000,0.000000
9,0.000000,0.445671,0.000000,0.000000,0.000000


# **4. Technical Reflection (2 points)**
- Contrast use cases for each technique -
  * *one-hot encoding reveals* that the top 5 "important" words were: "data", "also", "esa", "products", "project". These words shows up the most consistently across the 10 documents (descriptions).
  * *bag of words matrix* reveals that the top 5 "important" words were: "data", "dataset", "samples", "also", "cloud". These are the highest reoccurring words in the 10 documents. For words such as "sample" and "cloud" they did not show up very consistently across all 10 documents but there were enough reoccurrences in a mere few documents (descriptions) for them to become one of the top 5 words in BoW.
  * TF-IDF matrix reveals that the top 5 "important" words were: "data", "samples", "dataset", "cloud", "products". These were words that either were frequently showing up or words that rarely showed up across all documents but when they did show up, they showed up a lot.
- Explain how TF-IDF solves BoW limitations - BoW scores were high for stopwords that occur frequently but weren't important. TF-IDF resolved this by assigning lower scores to words that occurred frequently across ALL documents (descriptions). This allows words like "also" to not have a higher score than say, "cloud".

- Discuss one challenge with sparse representations - One challenge with sparse representations is it's limited capturing of context. This means that these representation can't capture the true meaning behind words and thus the usages of these encodings are severely limited compared to under text representations.
- Suggest an AI prompt for handling large vocabularies - "How can I prevent my computer from running out of RAM when calculating the TF-IDF, BoW, or one-hot encodings from a corpus has a large vocabulary?"